# Discretizations used in EllipSys

The "vis" variable in the EllipSys code is the *dynamic eddy viscosity", $\mu_t$.

## Segregated solver for 1D steady-state

The procedure of obtaining a steady-state solution is segregated:

- Solve $u$-mom'm equation using most recent values of $\nu_t$
- Solve $v$-mom'm eq.
- Solve $k$-eq.
- Solve $\varepsilon$-eq.
- Calculate $\nu_t$.
- Repeat process, since the new $\nu_t$ means that $u$ is no longer a solution to the $u$-mom'm eq. Same goes for the other equations.


## Steady EllipSys1D $k$-equation

As an example consider the $k$-equation:

$$
0 = \vec{\nabla} \cdot \left( \underbrace{\left(\mu + \frac{\mu_t}{\sigma_k}\right)}_{\mu_a} \vec{\nabla}k \right)  + \rho \mathcal{P} - \rho \varepsilon
$$

Integrate over the cell volume:

$$
0 = \int_V \vec{\nabla} \cdot \left( \mu_a \vec{\nabla}k \right)dV  + \int_V\rho\mathcal{P}dV - \int_V\rho\varepsilon dV
$$


Apply divergence theorem:

$$
0 = \int_A  \mu_a \vec{\nabla}k \cdot d\vec{A}  + \int_V\rho\mathcal{P}dV - \int_V\rho\varepsilon dV
$$

In 1D the bottom and top faces have unit area. The volume in 1D is therefore just equal to $V = \Delta z$.

$$
\begin{align}
0 &= \left(\mu_a \frac{\partial k}{\partial z}\right)_{top} - \left(\mu_a \frac{\partial k}{\partial z}\right)_{bottom}   + \rho \mathcal{P} \Delta z - \rho\varepsilon \Delta z \\
  &= \mu_{a,top} \left(\frac{k_{i+1} - k_{i}}{\Delta z}\right) - \mu_{a,bottom} \left(\frac{k_{i} - k_{i-1}}{\Delta z}\right)   + \rho \mathcal{P} \Delta z - \rho\varepsilon \Delta z \\
  &= \frac{\mu_{a,bottom}}{\Delta z} k_{i-1} - \frac{\mu_{a,bottom} + \mu_{a,top}}{\Delta z} k_{i} + \frac{\mu_{a,top}}{\Delta z} k_{i+1}  + \rho\mathcal{P} \Delta z - \rho\varepsilon \Delta z \\
\end{align}
$$

Move diffusion and dissipation to LHS:
$$
\begin{align}
\underbrace{-\frac{\mu_{a,bottom}}{\Delta z}}_{a_b} k_{i-1} + \underbrace{\left(\frac{\mu_{a,bottom} + \mu_{a,top}}{\Delta z} + \rho\varepsilon \Delta z \frac{1}{k_i}\right)}_{a_P} k_{i} \underbrace{- \frac{\mu_{a,top}}{\Delta z}}_{a_t} k_{i+1}  =    \underbrace{\rho\mathcal{P} \Delta z}_{s}  \\
\end{align}
$$

Hence, we have a matrix system:

$$
\begin{bmatrix} \ddots  & & &  \\ & a_b & a_P & a_t &  \\ & & & \ddots   \end{bmatrix} \begin{bmatrix} \vdots \\ k_i \\ \vdots   \end{bmatrix} = \begin{bmatrix} \vdots \\ s_i \\ \vdots   \end{bmatrix} 
$$

$$
A \vec{k} = \vec{s}
$$

- Can be solved with a tridiagonal solver to obtain $\vec{k}$.
- $\varepsilon > 0$ and moving it to the LHS makes the diagonal more positive.



## Rough wall boundary $k$-equation


<img src="rough_wall.png">


The $k$-equation is modified in the first cell adjacent to the wall (Sørensen, 2007):

- A "von Neumann" BC is applied at the first cell adjacent to the wall, i.e. $k_i = k_{i-1}$.
- The production is set to the cell-averaged equilibrium value, $\mathcal{P} \rightarrow \mathcal{P}_{log}$.

$$
\begin{align}
\mathcal{P}_{log} &\equiv \frac{1}{z_{n} - 0} \int_{0}^{z_{n}} - \overline{u w} \frac{\partial U}{\partial z} dz \\
&\approx \frac{u_*^2}{\Delta z} \int_{0}^{z_{n}}  \frac{\partial U}{\partial z} dz \\
&= \frac{u_*^2}{\Delta z} U(z_n) \\
&= \frac{u_*^2}{\Delta z} \frac{\mathrm{ln}(2 z_P/z_0 + 1)}{\mathrm{ln}(z_P/z_0 + 1)} U(z_p) \\
\end{align}
$$

- Notation compared to Sørensen (2007):
  - In this notebook $\Delta z$ is the cell height, while Sørensen (2007) use $\Delta z$ as half of the cell height.
  - In this notebook $\left[\mathcal{P}\right] = \frac{\textrm{m}^2}{\textrm{s}^3}$, while Sørensen (2007) use $\left[\mathcal{P}\right] = \frac{\textrm{kg}}{\textrm{m}^3} \frac{\textrm{m}^2}{\textrm{s}^3}$. Therefore, we have $u_*^2$ instead of $\tau_w$ in the above equation.

- Finally, the dissipation in the first cell is set to the cell-averaged equilibrium values, $\varepsilon \rightarrow \varepsilon_{log}$.


$$
\begin{align}
\mathcal{\varepsilon}_{log} &\equiv \frac{1}{z_{n} - 0} \int_{0}^{z_{n}} \varepsilon(z) dz \\
&\approx \frac{u_*^3}{\Delta z \kappa } \int_{0}^{z_{n}}  \frac{1}{(z + z_0)} dz \\
&= \frac{u_*^3}{\Delta z \kappa} \left(U(z_n+z_0) - U(z_0) \right) \\
&= \frac{u_*^2}{\Delta z} U(z_n) \\
&= \mathcal{P}_{log} \\
\end{align}
$$

- We have a problem! The production and dissipation just cancels. We wish to force $k$ towards equilibrium instead. In the above we assumed $u_* = \sqrt{-\overline{u w}}$ is constant throughout the cell, which also corresponds to saying $k$ is assumed constant throughout the cell, since $k_{eq} = u_*^2/C_\mu^{1/2}$. To "force" $k_i$ towards this, we do:
$$
\begin{align}
\varepsilon_{i} &= \mathcal{P}_{log} \frac{k_P}{k_{eq}} \\
&= \mathcal{P}_{log} \frac{C_\mu^{1/2}}{u_*^2} k_P \\
&= \frac{1}{\Delta z} \underbrace{\frac{\mathrm{ln}(2 z_P/z_0 + 1)}{\mathrm{ln}(z_P/z_0 + 1)} U(z_p)}_{"const"} C_\mu^{1/2} k_P \\
&= \frac{const}{\Delta z} C_\mu^{1/2} k_P
\end{align}
$$

**Not sure if I understood $\varepsilon_{log}$ correct, ask Niels S**.

The $k$-symmetry BC leads to the following equation:


$$
\begin{align}
0  &= \mu_{a,top} \left(\frac{k_{i+1} - k_{i}}{\Delta z}\right) - \mu_{a,bottom} \left(\frac{k_{i} - k_{i-1}}{\Delta z}\right)   + \rho \mathcal{P} \Delta z - \rho\varepsilon \Delta z \\
   &= \mu_{a,top} \left(\frac{k_{i+1} - k_{i}}{\Delta z}\right)    + \rho \mathcal{P} \Delta z - \rho\varepsilon \Delta z \\
  &=  - \frac{\mu_{a,top}}{\Delta z} k_{i} + \frac{\mu_{a,top}}{\Delta z} k_{i+1}  + \rho\mathcal{P} \Delta z - \rho\varepsilon \Delta z \\
\end{align}
$$

Move diffusion and dissipation to LHS:
$$
\begin{align}
\left(\frac{- \mu_{a,top}}{\Delta z} + \rho\varepsilon \Delta z \frac{1}{k_i}\right) k_{i} - \frac{\mu_{a,top}}{\Delta z} k_{i+1}  =    \underbrace{\rho\mathcal{P} \Delta z}_{s}  \\
\end{align}
$$

Insert cell-averaged production and dissipation. Identify coefficients:

$$
\begin{align}
\Rightarrow \underbrace{0}_{a_b} k_{i-1} + \underbrace{\left(\frac{0 + \mu_{a,top}}{\Delta z} + \rho C_\mu^{1/2} const \right)}_{a_P} k_{i} \underbrace{- \frac{\mu_{a,top}}{\Delta z}}_{a_t} k_{i+1}  =    \underbrace{\rho\mathcal{P}_{log} \Delta z}_{s}  \\
\end{align}
$$







## Adding buoyancy term

The RHS now becomes:

$$
RHS =  \underbrace{\rho\mathcal{P} \Delta z + \rho \mathcal{B} \Delta z}_{s}
$$

### Density based (default EllipSys)

Temperature equation is solved for $T$. Next obtain density through ideal gas law in molar mass form (https://en.wikipedia.org/wiki/Ideal_gas_law):

$$
\rho = \frac{p_{atm} M}{R T}
$$

Atmospheric pressure $p_{atm} = 1 \cdot 10^5 ~\textrm{Pa}$, molar mass of air $M = 0.028~\frac{\mathrm{kg}}{\mathrm{mol}}$ and universal gas constant $R =8.313~\frac{\mathrm{m}^3~\mathrm{Pa}}{\mathrm{mol}~\mathrm{K}}$. Then calculate buoyancy production as:

$$
\begin{align}
\mathcal{B} &= - \frac{g}{\rho_0} \overline{w'\rho'} \\
&= - \frac{g}{\rho_0} \left( -\frac{\nu_t}{Pr_\rho} \frac{d \rho}{dz} \right)
\end{align}
$$

Discretization:

$$
\left(\rho \mathcal{B} \Delta z \right)_i = \frac{g}{2 \rho_0 Pr_\rho} \left( \mu_t\right)_i \left( \rho_{i+1} -\rho_{i-1} \right)
$$

*For some reason, there is no $\rho_0$ and an extra sign in the code. This will make $\mathcal{B} > 0$ for stable, which I think is wrong.* $g = 9.81~\mathrm{m}/\mathrm{s}^2$ and $Pr_\rho = 1$ is default, which was also used by for example Gemmell (2021).



### Temperature based

Temperature equation is solved for $\Theta$. Obtain the buoyancy production directly as:

$$
\begin{align}
\mathcal{B} &= \frac{g}{\theta_0} \overline{w'\theta'} \\
&= \frac{g}{\theta_0} \left( -\frac{\nu_t}{Pr_t} \frac{d \Theta}{dz} \right)
\end{align}
$$

Here $\theta_0$ is a "reference potential temperature". It is constant for neutral ABL, while it in principle varies with $z$ in non-neutral ABL, but it is an okay assumption to set $\theta_0(z) \approx \theta_0(0)$.

Discretization:

$$
\left(\rho \mathcal{B} \Delta z \right)_i = - \frac{g}{2 \theta_0 Pr_t} \left( \mu_t\right)_i \left( \Theta_{i+1} -\Theta_{i-1} \right)
$$



## Adding unsteady term

Implicit Euler scheme (aka. "backward Euler"):

$$
\frac{d k}{d t} = \frac{k^{(n+1)} - k^{(n)}}{\Delta t} = f(k^{(n+1)})
$$

$$
\Rightarrow k^{(n+1)}   = f(k^{(n+1)}) \Delta t + k^{(n)}
$$

, where $f$ symbolizes the diffusion, production and dissipation evaluated at timestep $n+1$. Since $f(k^{(n+1)}$ is not known, we use "subiterations":

- To advance from step $n$ to $n+1$, use $f(k^{(n)})$ instead of $f(k^{(n+1)})$ for a start.
- Obviously this gives the wrong $k^{(n+1)}$, so let's call this solution "$k^*$" instead.
- Now give it another go the integration another go, but now using $f(k^*)$. Again, this does not give $k^{(n+1)}$, but something else; let's call the solution $k^{*}$ (i.e. overwrite old $k^*$).
- This procedure can be continued and the number of repetitions is called the "number of subiterations". The idea is that $k* \rightarrow k^{(n+1)}$ after enough subiterations.
- Notice, that the second term on the RHS, "$+k^{(n)}$", is kept constant throughout the subiterations.

Remember, that we multiply by $\rho$ and integrate over the volume to obtain the A matrix:


$$
\begin{align}
\int_V \rho \frac{d k}{d t} dV &= \rho \frac{d k}{d t} \Delta z \\
                               &=  \frac{\rho \Delta z}{\Delta t} k^{(n+1)} - \frac{\rho \Delta z}{\Delta t} k^{(n)} 
\end{align}
$$

It is clear that the $\frac{d k}{d t}$ does *not* depend on neighbouring cells! It will only affect the diagonal and source term:

$$
\begin{bmatrix} \ddots  & & &  \\ & a_b & a_P + \frac{\rho \Delta z}{\Delta t} & a_t &  \\ & & & \ddots   \end{bmatrix} \begin{bmatrix} \vdots \\ k_i^{(n+1)} \\ \vdots   \end{bmatrix} = \begin{bmatrix} \vdots \\ s_i +  \frac{\rho \Delta z}{\Delta t} k_i^{(n)}\\ \vdots   \end{bmatrix} 
$$



## Underrelaxation in steady 1D

Idea: Instead of providing $\vec{k}$ (will be used in the solution of $\varepsilon$-, $u$- and $v$-equations, see section about segregated solver above), we can mix it up with some of the old $\vec{k}$ from the previous iteration. This has benefits for numerical stability and the final solution will not be affected by this (Patankar 1980, p.67). One solves the following system instead (Patankar 1980, p.67):


$$
\begin{bmatrix} \ddots  & & &  \\ & a_b & \dfrac{a_P}{\alpha} & a_t &  \\ & & & \ddots   \end{bmatrix} \begin{bmatrix} \vdots \\ k_i \\ \vdots   \end{bmatrix} = \begin{bmatrix} \vdots \\ s_i + (1-\alpha)\dfrac{a_P}{\alpha} k_i^{(old)} \\ \vdots   \end{bmatrix} 
$$

## Underrelaxation in unsteady 1D

Same formula as in "underrelaxation in steady 1D", but where the extra "unsteady" bit of the diagonal has been pulled into $a_P$. Same with source term on the RHS.
